In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.metrics import recall_score
from model import ready_model

f:\anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
f:\anaconda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
f:\anaconda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
kfold = KFold(n_splits=5, shuffle=True)
data = pd.read_table("results.csv",delimiter=",",skiprows=0, header = None)
X, Y = np.array(data.iloc[:,1:-1]),np.array(data.iloc[:,-1])

In [3]:
def get_model_name(k, name):
    return 'model_'+name+str(k)+'.h5'

In [4]:
name = "crossval"
accuracy = []
recall = []
fold_no = 1
for train, test  in kfold.split(X, Y):
    train_model = ready_model()
    train_model.compile(optimizer = Adam(learning_rate=0.0001), 
                        loss = SparseCategoricalCrossentropy(),
                        metrics = ['accuracy'])
    print(f'training fold number {fold_no}')

    save_dir = 'model/crossval/'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_no, name), 
            monitor='val_accuracy', verbose=1, 
            save_best_only=True, mode='max')

    callbacks_list = [checkpoint]

    history = train_model.fit(X[train], Y[train], epochs = 60, 
                            validation_data = (X[test], Y[test]), 
                            batch_size = 64,
                            callbacks = callbacks_list,
                            verbose = 0)


    train_model.load_weights(save_dir+get_model_name(fold_no, name))
    scores = train_model.evaluate(X[test], Y[test], verbose=0)
    accuracy.append(scores[1])
    y_pred = train_model.predict(X[test])
    y_preds_ready = y_pred.argmax(axis=1)
    recall.append(recall_score(Y[test], y_preds_ready, average='macro'))
    fold_no+=1
print("score per fold:")
for i in range(5):
    print(f'Accuracy in fold {i+1}: {accuracy[i]*100}, Recall in fold {i+1} : {recall[i]*100}')

print(f"the Averrage Accuracy score for 5 fold cross validation is {sum(accuracy)/5 *100} and recall is {sum(recall)/5*100}")





training fold number 1

Epoch 00001: val_accuracy improved from -inf to 0.31579, saving model to model/crossval\model_crossval1.h5

Epoch 00002: val_accuracy did not improve from 0.31579

Epoch 00003: val_accuracy did not improve from 0.31579

Epoch 00004: val_accuracy did not improve from 0.31579

Epoch 00005: val_accuracy did not improve from 0.31579

Epoch 00006: val_accuracy did not improve from 0.31579

Epoch 00007: val_accuracy did not improve from 0.31579

Epoch 00008: val_accuracy did not improve from 0.31579

Epoch 00009: val_accuracy did not improve from 0.31579

Epoch 00010: val_accuracy did not improve from 0.31579

Epoch 00011: val_accuracy did not improve from 0.31579

Epoch 00012: val_accuracy did not improve from 0.31579

Epoch 00013: val_accuracy did not improve from 0.31579

Epoch 00014: val_accuracy improved from 0.31579 to 0.45614, saving model to model/crossval\model_crossval1.h5

Epoch 00015: val_accuracy improved from 0.45614 to 0.53070, saving model to model/cro

In [5]:
accuracy

[1.0, 1.0, 1.0, 0.9956140518188477, 1.0]

In [6]:
recall

[1.0, 1.0, 1.0, 0.995, 1.0]